In [1]:
!pip install -q kaggle

In [2]:
!mkdir ~/.kaggle


In [3]:
ls~/.kaggle

In [4]:
!kaggle datasets download -d kalyankaparaju01/credit-card-fraud-detection-dataset

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'


In [5]:
import zipfile
with zipfile.ZipFile("credit-card-fraud-detection-dataset.zip","r") as file:
    file.extractall()

FileNotFoundError: [Errno 2] No such file or directory: 'credit-card-fraud-detection-dataset.zip'

In [ ]:
%pip install seaborn

In [ ]:
%pip install matplotlib

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
from sklearn.model_selection import GridSearchCV

In [ ]:
# Model Evaluation Modules
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, accuracy_score, f1_score, r2_score
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.metrics import recall_score

In [ ]:
df = pd.read_csv('fraudTrain.csv')
df.head()

In [ ]:
len(df)

In [ ]:
#dtypes of the columns
df.dtypes

In [ ]:
#converting trans_date_trans_time into datetime
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
print(df.dtypes['trans_date_trans_time'])


In [ ]:
# deriving additonal columns from 'trans_date_trans_time'
#deriving hour
df['trans_hour'] = df['trans_date_trans_time'].dt.hour
#deriving 'day of the week
df['trans_day_of_week'] = df['trans_date_trans_time'].dt.dayofweek + 1
df['trans_day_of_week'] = df['trans_day_of_week'].astype(int)
#deriving 'year_month'
df['trans_year_month'] = df['trans_date_trans_time'].dt.to_period('M')

In [ ]:
#converting data types that should be categorical into "category"

df['category'] = df['category'].astype('category')
df['gender'] = df['gender'].astype('category')
df['is_fraud'] = df['is_fraud'].astype('category')

In [ ]:
df.dtypes

In [ ]:
#dropping variables which are not usefull for the Visualization and analytics
df.drop(['trans_date_trans_time','first', 'last', 'dob',] , axis=1, inplace=True)

In [ ]:
df.info()

In [ ]:
#let us look at the number of unique values in the dataset
df.nunique()

In [ ]:
df.sample()

In [ ]:
#determing the shape of the dataset
df.shape

In [ ]:
df.columns

In [ ]:
#describing the dataset
df.describe()

In [ ]:
# checking the number of missing values in each column
df.isnull().sum()

In [ ]:
# distribution of legit transactions & fraudulent transactions
df['is_fraud'].value_counts()

In [ ]:
#plotting the above distributions
fig = plt.subplots(figsize=(15,10))

plots = []
#plotting the amt feature

#distribution plots
plots.append(sns.histplot(df[df.amt <= 1500].amt, bins=50, ax=plt.subplot(234)))
plots.append(sns.histplot(df[(df.is_fraud==0) & (df.amt<=1500)].amt, bins=50, ax=plt.subplot(235)))
plots.append(sns.histplot(df[(df.is_fraud==1) & (df.amt<=1500)].amt, bins=50, ax=plt.subplot(236)))

#setting titles
plots[0].set_title('Overall Amount Distribution')
plots[1].set_title('Non Fraud Amount Distribution')
plots[2].set_title('Fraud Amount Distribution')

#setting x labels
plots[0].set_xlabel('Transaction Amount')
plots[1].set_xlabel('Transaction Amount')
plots[2].set_xlabel('Transaction Amount')

#setting y label
plots[0].set_ylabel('Number of transactions')

plt.show()



In [ ]:
#year_month vs number of transactions
df_timeline01 = df.groupby(df['trans_year_month'])[['trans_num','cc_num']].nunique().reset_index()
df_timeline01.columns = ['year_month','num_of_transactions','customers']
df_timeline01

In [ ]:
# Set up the plot for is_fraud = 1
fig, ax1 = plt.subplots(figsize=(14, 8))

# Get the order of states by their count in descending order
state_order = df[df['is_fraud'] == 1]['state'].value_counts().index

# Plotting the count of each state for is_fraud = 1
plot1 = sns.countplot(x='state', data=df[df['is_fraud'] == 1], order=state_order, palette='Set2', ax=ax1)
plot1.set_xticklabels(plot1.get_xticklabels(), rotation=90)

ax1.set_title('Transaction Count by State in a fraudulent transaction (Descending Order)')

# Show the plot for is_fraud = 1
plt.show()

In [ ]:
# Calculate fraud ratios for all states
fraud_ratios = []

for state in df['state'].unique():
    total_count = df[df['state'] == state].shape[0]
    fraud_count = df[(df['state'] == state) & (df['is_fraud'] == 1)].shape[0]
    ratio = fraud_count / total_count if total_count > 0 else 0
    fraud_ratios.append({'State': state, 'Fraud Ratio': ratio, 'Total Transactions': total_count})

# Create a DataFrame from the list of dictionaries
fraud_ratio_df = pd.DataFrame(fraud_ratios)

# Print the DataFrame
print(fraud_ratio_df)

In [ ]:
sns.countplot(x='is_fraud', data = df)
plt.title('Number of fraud vs non-fraud transcations')
plt.show()

In [ ]:
# separating the data for analysis
legit = df[df.is_fraud == 0]
fraud = df[df.is_fraud == 1]

In [ ]:
print(legit.shape)
print(fraud.shape)

In [ ]:
fraud['is_fraud'].value_counts()

In [ ]:
legit['is_fraud'].value_counts()

In [ ]:
fnew = legit.sample(n=6006)

In [ ]:
d_new = pd.concat([fnew, fraud], axis=0)

In [ ]:
d_new['is_fraud'].value_counts()

In [ ]:
d_new.shape

In [ ]:
sns.countplot(x='is_fraud', data = d_new)
plt.title('Number of fraud vs non-fraud transcations')
plt.show()

In [ ]:
X =  [ 'amt', 'gender','lat', 'long', 'city_pop',  'trans_hour','trans_day_of_week','is_fraud']
d_new= pd.get_dummies(d_new[X], drop_first=True)
predictors =  [ 'amt',  'city_pop', 'trans_hour', 'gender_M','lat', 'long','trans_day_of_week']

In [ ]:
# partition data
X = d_new[predictors]
Y = d_new[ 'is_fraud_1']

In [ ]:
# Splitting data into train and test set 80:20
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=100)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)


In [ ]:
print(X)

In [ ]:
# Impoting metrics
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, f1_score, precision_score, recall_score
results = pd.DataFrame(columns=['Model Name', 'Accuracy', 'F1-score', 'ROC','precision','recall'])

In [ ]:
# ROC Curve function

def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs,
                                              drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(5, 5))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    return None

In [ ]:
# Importing scikit logistic regression module
from sklearn.linear_model import LogisticRegression

In [ ]:
# Instantiate the model with best C
logistic = LogisticRegression(C=0.01)

In [ ]:
# Fit the model on the train set
logistic_model = logistic.fit(X_train, Y_train)

In [ ]:
# Prepare results function
def display_test_results(model_name, model):

    # Prediction on the test set
    Y_test_pred = model.predict(X_test)

    # Confusion matrix
    print("------------------ Confusion Matrix --------------------")
    c_matrix = metrics.confusion_matrix(Y_test, Y_test_pred)
    print(c_matrix)

    cm_display = ConfusionMatrixDisplay(confusion_matrix=c_matrix)
    cm_display.plot(cmap=plt.cm.Blues)
    plt.show()


    # classification_report
    print("------------------ classification_report --------------------")
    print(classification_report(Y_test, Y_test_pred))

    print("------------------ More Specific classification_report --------------------")
    TP = c_matrix[1,1] # true positive
    TN = c_matrix[0,0] # true negatives
    FP = c_matrix[0,1] # false positives
    FN = c_matrix[1,0] # false negatives

    # Accuracy
    print("Accuracy:-",metrics.accuracy_score(Y_test, Y_test_pred))

    # Sensitivity
    print("Sensitivity:-",TP / float(TP+FN))

    # Specificity
    print("Specificity:-", TN / float(TN+FP))

    # F1 score
    print("F1-Score:-", f1_score(Y_test, Y_test_pred))

    # Predicted probability
    Y_test_pred_proba = model.predict_proba(X_test)[:,1]

    # roc_auc
    print("------------------ ROC --------------------")
    roc_auc = metrics.roc_auc_score(Y_test, Y_test_pred_proba)

    # Plot the ROC curve
    draw_roc(Y_test, Y_test_pred_proba)

       # add all metrics score in final result store
    results.loc[len(results)] = [model_name, metrics.accuracy_score(Y_test, Y_test_pred), f1_score(Y_test, Y_test_pred), precision_score(Y_test, Y_test_pred),
    recall_score(Y_test, Y_test_pred),roc_auc]

    return None

In [ ]:
display_test_results("Logistic Regression", logistic_model)

In [ ]:
# Importing random forest classifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
random_forest_model = RandomForestClassifier(bootstrap=True,
                             max_depth=5,
                             min_samples_leaf=50,
                             min_samples_split=50,
                             max_features=10,
                             n_estimators=100)

In [ ]:
# Fit the model
random_forest_model.fit(X_train, Y_train)

In [ ]:
display_test_results("Random Forest", random_forest_model)

In [ ]:
# Importing decision tree classifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Model with optimal hyperparameters
decision_tree_model = DecisionTreeClassifier(criterion = "gini",
                                  random_state = 100,
                                  max_depth=5,
                                  min_samples_leaf=100,
                                  min_samples_split=100)

decision_tree_model.fit(X_train, Y_train)

In [ ]:
display_test_results("Decision Tree", decision_tree_model)

In [ ]:
results.sort_values(by="ROC", ascending=False)

In [ ]:
# Function to predict if a new transaction is fraud or not based on user input
def predict_fraud(amt, gender, lat, long, city_pop, trans_hour, trans_day_of_week):
    # Convert gender to numerical value, assuming "M" = 1 and "F" = 0
    gender_M = 1 if gender.lower() == "m" else 0

    # Create an input data array with all the required features
    input_data = np.array([[amt, city_pop, trans_hour, gender_M, lat, long, trans_day_of_week]])

    # Predict using the logistic regression model
    prediction = logistic_model.predict(input_data)
    prediction_proba = logistic_model.predict_proba(input_data)[0][1]  # Probability of fraud

    # Display the result
    if prediction[0] == 1:
        print(f"The transaction is likely to be FRAUDULENT with a probability of {prediction_proba:.2f}")
    else:
        print(f"The transaction is likely to be LEGITIMATE with a probability of {1 - prediction_proba:.2f}")

# Example user input for testing
# Note: Replace these values with real user input in a production setting
amt = float(input("Enter transaction amount: "))
gender = input("Enter gender (M/F): ")
lat = float(input("Enter latitude: "))
long = float(input("Enter longitude: "))
city_pop = int(input("Enter city population: "))
trans_hour = int(input("Enter transaction hour (0-23): "))
trans_day_of_week = int(input("Enter transaction day of week (1=Monday, 7=Sunday): "))

# Predict fraud based on input data
predict_fraud(amt, gender, lat, long, city_pop, trans_hour, trans_day_of_week)